### Imports

In [1]:
from ase_interface import ANIENS
from ase_interface import ensemblemolecule_multigpu
import hdnntools as hdt

import numpy as np
import ase

import time

import os
os.environ["OMP_NUM_THREADS"] = "4"

import  ase
from ase.md.langevin import Langevin
from ase.md.verlet import VelocityVerlet
from ase.io.trajectory import Trajectory
from ase import units
from ase.optimize import LBFGS
from ase.io import read, write
from ase.parallel import world
from ase import Atom, Atoms

import nglview as nv
from nglview.datafiles import PDB, XTC
import mdtraj as md
import pytraj as pt
from IPython.display import Image
from IPython.core.display import HTML 
from IPython.display import display, HTML

### Setup files to load (2LUF or TrpCage)

In [2]:
wkdir = '/home/jsmith48/scratch/MD_TEST/2luf/'

fname = '2luf_solv'

# Molecule file
molfile = wkdir + fname + '.pdb'

# Dynamics file
xyzfile = wkdir + 'mdcrd_qm9.xyz'

# Trajectory file
trajfile = wkdir + 'traj.dat'

# Optimized structure out:
optfile = wkdir + 'opt_' +  fname +'.xyz'

dt = 0.25
C = 0.1 # Optimization convergence

ntdir = '/home/jsmith48/scratch/qm9/model/'
cns = ntdir + 'rHCNOSFCl-5.2R_16-3.8A_a4-8.params'
sae = ntdir + 'sae_linfit.dat'
nnf = ntdir + 'train'
Nn = 8

### Load ANI models

In [3]:
# Load molecule
mol = read(molfile)

# Setup NeuroChem
aens = ensemblemolecule_multigpu(cns, sae, nnf, Nn, [4,5])

# Set ANI calculator
mol.set_calculator(ANIENS(aens))

# Set PBC
mol.set_cell(([[39.5, 0, 0],
               [0, 39.5, 0],
               [0, 0, 39.5]]))

mol.set_pbc((True, True, True))

Building...
Building...


Process Process-2:
Process Process-3:
Traceback (most recent call last):
KeyboardInterrupt
Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jsmith48/NeuroChem/build/lib/ase_interface.py", line 170, in molecule_worker
    next_task = task_queue.get()
  File "/home/jsmith48/NeuroChem/build/lib/ase_interface.py", line 170, in molecule_worker
    next_task = task_queue.get()
  File "/usr/local/anaconda3/lib/python3.6/multiprocessing/queues.py", line 94, in get
    res = self._recv_bytes()
  File "/usr/local/a

### Calculate energy and force

In [4]:
s_time = time.time()
E = mol.get_potential_energy()
F = mol.get_forces()
print('Total time:',time.time()-s_time)

print('Energy:',E)

spc = mol.get_chemical_symbols()
print('Number of atoms:',len(spc))

Total time: 2.787203073501587
Energy: -4658031.270077064
Number of atoms: 6732


### Optimize the system

In [5]:
# Optimize molecule
start_time = time.time()
dyn = LBFGS(mol)
dyn.run(fmax=C,steps=500)
print('[ANI Total time:', time.time() - start_time, 'seconds]')

# Save optimized mol
spc = mol.get_chemical_symbols()
pos = mol.get_positions(wrap=False).reshape(1,len(spc),3)


hdt.writexyzfile(optfile, pos, spc)

       Step     Time          Energy         fmax
LBFGS:    0 13:36:12 -4658031.270080        3.0081
LBFGS:    1 13:36:12 -4658063.098966        2.4735
LBFGS:    2 13:36:12 -4658030.894669        2.6684
LBFGS:    3 13:36:12 -4658016.883835        2.5531
LBFGS:    4 13:36:12 -4658012.271969        2.5367
LBFGS:    5 13:36:13 -4658012.778694        2.4861
LBFGS:    6 13:36:13 -4658011.430218        2.5380
LBFGS:    7 13:36:13 -4658009.582104        2.5411
LBFGS:    8 13:36:13 -4658006.984117        2.6150
LBFGS:    9 13:36:13 -4658003.397217        2.7732
LBFGS:   10 13:36:13 -4657998.928744        2.9006
LBFGS:   11 13:36:13 -4657997.953749        2.8786
LBFGS:   12 13:36:14 -4657999.514223        2.8951
LBFGS:   13 13:36:14 -4657999.115877        2.8783
LBFGS:   14 13:36:14 -4657999.854073        2.8226
LBFGS:   15 13:36:14 -4658002.007909        2.7051
LBFGS:   16 13:36:14 -4658005.469906        2.5269
LBFGS:   17 13:36:14 -4658002.522858        2.5656
LBFGS:   18 13:36:14 -4658000.48

LBFGS:  160 13:36:35 -4657913.899440       12.7502
LBFGS:  161 13:36:35 -4657914.128805       11.6130
LBFGS:  162 13:36:35 -4657915.336121       11.7317
LBFGS:  163 13:36:35 -4657916.949670       12.0056
LBFGS:  164 13:36:36 -4657918.534544       11.1675
LBFGS:  165 13:36:36 -4657920.053967       10.2338
LBFGS:  166 13:36:36 -4657918.896103       12.2473
LBFGS:  167 13:36:36 -4657920.758733       10.1219
LBFGS:  168 13:36:36 -4657919.015778       10.2190
LBFGS:  169 13:36:36 -4657918.778108        9.4942
LBFGS:  170 13:36:37 -4657920.631105        9.8393
LBFGS:  171 13:36:37 -4657922.209694       10.3593
LBFGS:  172 13:36:37 -4657924.079368       10.1529
LBFGS:  173 13:36:37 -4657925.758157       10.4975
LBFGS:  174 13:36:37 -4657926.020318       11.3925
LBFGS:  175 13:36:37 -4657924.385231       12.0707
LBFGS:  176 13:36:37 -4657926.011206       11.3112
LBFGS:  177 13:36:38 -4657928.098280       10.2184
LBFGS:  178 13:36:38 -4657930.087458        9.5103
LBFGS:  179 13:36:38 -4657932.1

LBFGS:  321 13:36:59 -4658036.867130       23.5815
LBFGS:  322 13:36:59 -4658042.938706       25.6858
LBFGS:  323 13:36:59 -4658048.389338       25.5094
LBFGS:  324 13:36:59 -4658054.167144       25.5513
LBFGS:  325 13:37:00 -4658059.532944       25.4863
LBFGS:  326 13:37:00 -4658064.869676       25.1429
LBFGS:  327 13:37:00 -4658069.901766       24.5263
LBFGS:  328 13:37:00 -4658074.956071       23.6056
LBFGS:  329 13:37:00 -4658079.955321       22.5216
LBFGS:  330 13:37:00 -4658084.645174       22.5054
LBFGS:  331 13:37:01 -4658089.702362       22.1734
LBFGS:  332 13:37:01 -4658094.227182       21.0911
LBFGS:  333 13:37:01 -4658098.554827       19.9572
LBFGS:  334 13:37:01 -4658102.705931       20.3927
LBFGS:  335 13:37:01 -4658106.213251       28.6917
LBFGS:  336 13:37:01 -4658109.698114       38.0810
LBFGS:  337 13:37:01 -4658109.338222       38.1548
LBFGS:  338 13:37:02 -4658110.684962       39.0150
LBFGS:  339 13:37:02 -4658115.321592       33.8507
LBFGS:  340 13:37:02 -4658120.8

LBFGS:  482 13:37:24 -4658504.804784       97.7507
LBFGS:  483 13:37:24 -4658507.443974       98.1321
LBFGS:  484 13:37:24 -4658510.130731       97.7997
LBFGS:  485 13:37:24 -4658512.785383       99.0064
LBFGS:  486 13:37:24 -4658515.415937      101.4533
LBFGS:  487 13:37:25 -4658517.980757      103.9161
LBFGS:  488 13:37:25 -4658521.126703      102.8787
LBFGS:  489 13:37:25 -4658523.394955      103.3948
LBFGS:  490 13:37:25 -4658525.809249      103.2311
LBFGS:  491 13:37:25 -4658526.424221      105.2082
LBFGS:  492 13:37:25 -4658526.962997      104.4290
LBFGS:  493 13:37:25 -4658529.860452      103.1306
LBFGS:  494 13:37:26 -4658531.892831      101.3673
LBFGS:  495 13:37:26 -4658534.463857       98.0553
LBFGS:  496 13:37:26 -4658537.860006       94.4500
LBFGS:  497 13:37:26 -4658541.157677       88.6235
LBFGS:  498 13:37:26 -4658544.069948       86.5245
LBFGS:  499 13:37:26 -4658546.790754       85.9825
[ANI Total time: 74.33779764175415 seconds]


### Setup thermostat

In [6]:
# We want to run MD with constant energy using the VelocityVerlet algorithm.
# coefficient to 0.04 atomic units.
dyn = Langevin(mol, dt * units.fs, 1.0 * units.kB, 0.04)

In [7]:
# Open MD output
mdcrd = open(xyzfile,'w')

# Open MD output
traj = open(trajfile,'w')

# Define the printer
def storeenergy(a=mol, d=dyn, b=mdcrd, t=traj):  # store a reference to atoms in the definition.
    """Function to print the potential, kinetic and total energy."""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)

    t.write(str(d.get_number_of_steps()) + ' ' + str(ekin / (1.5 * units.kB)) + ' ' + str(epot) + ' ' + str(ekin) + ' ' + str(epot+ekin) + '\n')
    b.write(str(len(a)) + '\n' + str(ekin / (1.5 * units.kB)) + ' Step: ' + str(d.get_number_of_steps()) + '\n')
    c = a.get_positions(wrap=False)
    for j, i in zip(a, c):
        b.write(str(j.symbol) + ' ' + str(i[0]) + ' ' + str(i[1]) + ' ' + str(i[2]) + '\n')

    print('Step: %d - %.1f ps Energy per atom: Epot = %.3f  Ekin = %.3f (T=%.3fK)  '
          'Etot = %.6f' % (d.get_number_of_steps(), (d.get_number_of_steps()*0.25)/1000, epot, ekin, ekin / (1.5 * units.kB), epot + ekin))
    
# Set the printer
dyn.attach(storeenergy, interval=25)

### Run MD

In [8]:
start_time = time.time()
dyn.set_temperature(300. * units.kB)
dyn.run(1000000)
print('[ANI Total time:', time.time() - start_time, 'seconds]')

Step: 25 - 0.0 ps Energy per atom: Epot = -692.053  Ekin = 0.053 (T=407.724K)  Etot = -692.000089
Step: 50 - 0.0 ps Energy per atom: Epot = -692.175  Ekin = 0.171 (T=1324.138K)  Etot = -692.003981
Step: 75 - 0.0 ps Energy per atom: Epot = -692.248  Ekin = 0.235 (T=1819.474K)  Etot = -692.012799
Step: 100 - 0.0 ps Energy per atom: Epot = -692.296  Ekin = 0.272 (T=2100.909K)  Etot = -692.024010
Step: 125 - 0.0 ps Energy per atom: Epot = -692.353  Ekin = 0.317 (T=2449.426K)  Etot = -692.036124
Step: 150 - 0.0 ps Energy per atom: Epot = -692.419  Ekin = 0.362 (T=2797.415K)  Etot = -692.057316
Step: 175 - 0.0 ps Energy per atom: Epot = -692.478  Ekin = 0.408 (T=3157.577K)  Etot = -692.069724
Step: 200 - 0.1 ps Energy per atom: Epot = -692.531  Ekin = 0.443 (T=3429.851K)  Etot = -692.087183
Step: 225 - 0.1 ps Energy per atom: Epot = -692.640  Ekin = 0.531 (T=4107.014K)  Etot = -692.109424
Step: 250 - 0.1 ps Energy per atom: Epot = -692.724  Ekin = 0.589 (T=4553.882K)  Etot = -692.135125
Step

/usr/local/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:138: RuntimeWarning: invalid value encountered in sqrt
  ret = um.sqrt(ret, out=ret)


Step: 1475 - 0.4 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 1500 - 0.4 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 1525 - 0.4 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 1550 - 0.4 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 1575 - 0.4 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 1600 - 0.4 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 1625 - 0.4 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 1650 - 0.4 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 1675 - 0.4 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 1700 - 0.4 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 1725 - 0.4 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 1750 - 0.4 ps Energy per atom: Epot = -692.009  

Step: 3875 - 1.0 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 3900 - 1.0 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 3925 - 1.0 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 3950 - 1.0 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 3975 - 1.0 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 4000 - 1.0 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 4025 - 1.0 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 4050 - 1.0 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 4075 - 1.0 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 4100 - 1.0 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 4125 - 1.0 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 4150 - 1.0 ps Energy per atom: Epot = -692.009  

Step: 6275 - 1.6 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 6300 - 1.6 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 6325 - 1.6 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 6350 - 1.6 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 6375 - 1.6 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 6400 - 1.6 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 6425 - 1.6 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 6450 - 1.6 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 6475 - 1.6 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 6500 - 1.6 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 6525 - 1.6 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 6550 - 1.6 ps Energy per atom: Epot = -692.009  

...

Step: 13425 - 3.4 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 13450 - 3.4 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 13475 - 3.4 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 13500 - 3.4 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 13525 - 3.4 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 13550 - 3.4 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 13575 - 3.4 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 13600 - 3.4 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 13625 - 3.4 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 13650 - 3.4 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 13675 - 3.4 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 13700 - 3.4 ps Energy per atom: Epot 

Step: 15800 - 4.0 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 15825 - 4.0 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 15850 - 4.0 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 15875 - 4.0 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 15900 - 4.0 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 15925 - 4.0 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 15950 - 4.0 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 15975 - 4.0 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 16000 - 4.0 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 16025 - 4.0 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 16050 - 4.0 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 16075 - 4.0 ps Energy per atom: Epot 

Step: 18175 - 4.5 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 18200 - 4.5 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 18225 - 4.6 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 18250 - 4.6 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 18275 - 4.6 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 18300 - 4.6 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 18325 - 4.6 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 18350 - 4.6 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 18375 - 4.6 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 18400 - 4.6 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 18425 - 4.6 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 18450 - 4.6 ps Energy per atom: Epot 

Step: 20550 - 5.1 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 20575 - 5.1 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 20600 - 5.2 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 20625 - 5.2 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 20650 - 5.2 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 20675 - 5.2 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 20700 - 5.2 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 20725 - 5.2 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 20750 - 5.2 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 20775 - 5.2 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 20800 - 5.2 ps Energy per atom: Epot = -692.009  Ekin = nan (T=nanK)  Etot = nan
Step: 20825 - 5.2 ps Energy per atom: Epot 

KeyboardInterrupt: 

### Visualization

In [9]:
m_traj = md.load_xyz(wkdir+'mdcrd_qm9.xyz',top=wkdir + fname + '.pdb')
m_view = nv.show_mdtraj(m_traj)
m_view.center()
m_view.add_representation( "ball+stick", selection='protein')
m_view.add_licorice('water and not hydrogen')
m_view

A Jupyter Widget